In [89]:
import numpy as np
import re

In [91]:
Rs = [0.00013310329,8.8735527e-05,0.0002440227,0.00026620659,0.00044367764]
Rb = [2.2183882e-05,1e-06,2.2183882e-05,4.4367764e-05,8.8735527e-05]
As = [2.90476,1.21032,2.17857,2.6627,4.11508]
Ab = [20.5754,16.9445,18.881,22.754,35.5833]
Cs = [6,4,11,12,20]
Cb = [1,0,1,2,4]
Ts = [45077.8,45077.8,45077.8,45077.8,45077.8]
Tb = [45077.8,45077.8,45077.8,45077.8,45077.8]

In [93]:
Rs = np.array(Rs)
Rb = np.array(Rb)
As = np.array(As)
Ab = np.array(Ab)
Cs = np.array(Cs)
Cb = np.array(Cb)
Ts = np.array(Ts)
Tb = np.array(Tb)

In [95]:
Rnet = Rs - Rb*(As/Ab)

sigma_Rs = np.sqrt(Cs) / Ts
sigma_Rb = np.sqrt(Cb) / Tb

sigma_Rnet = np.sqrt(sigma_Rs**2 + (sigma_Rb * As / Ab)**2)

In [97]:
SNR = Rnet/sigma_Rnet

In [99]:
SNR

array([2.38789337, 1.99839125, 3.27985282, 3.39267212, 4.36287025])

In [101]:
absorved = 4.162E-12 
unabsorved = 5.185E-12  

flux_abs = Rnet * absorved
flux_abs_err = sigma_Rnet * absorved
flux_unabs = Rnet * unabsorved
flux_unabs_err = sigma_Rnet * unabsorved

In [103]:
def luminosidad(flux, distance, flux_error, distance_error):
    distance = distance * 3.086 * 10**24
    
    L = 4 * np.pi * distance**2 * flux
    L_error = np.sqrt((4 * np.pi * distance**2 * flux_error)**2 + (8 * np.pi * distance * flux * distance_error)**2)
    
    return L, L_error

dist =  8.63 # en Mpc
e_dist = 0.30

lumi_abs, lumi_abs_err = luminosidad(flux_abs, dist, flux_abs_err, e_dist)
lumi_unabs, lumi_unabs_err = luminosidad(flux_unabs, dist, flux_unabs_err, e_dist)

In [105]:
def cargar_coords(nombre_reg):
    x_list = []
    y_list = []
    r_list = []

    with open(nombre_reg, 'r') as f:
        for line in f:
            if line.startswith("circle("):
                # Cerca valors dins de circle(...)
                match = re.search(r'circle\(([^,]+),([^,]+),([^\)"]+)', line)
                if match:
                    x, y, r = map(float, match.groups())
                    x_list.append(x)
                    y_list.append(y)
                    r_list.append(r)
    
    return np.array(x_list), np.array(y_list), np.array(r_list)

In [107]:
path = 'wavdetect/broad_area_WCS.reg'
coords = cargar_coords(path)

In [123]:
x, y, r = coords
# Eliminar el primer elemento
x = x[1:]
y = y[1:]
r = r[1:]

print(x, y)

[24.1829813 24.1989824 24.1768352 24.1670393 24.1873178] [15.7993186 15.7906145 15.7835827 15.7736928 15.7628483]


In [125]:
import csv


energy = [['medium','soft'],['medium','soft'],['soft'],['medium','soft'],['hard','medium','soft']]

header = [
    'RA', 'Dec',
    '$F_{abs}$', '$F_{unabs}$',
    'flux_abs_err', 'flux_unabs_err',
    'log($L_{abs}$)', 'log10($L_{unabs}$)',
    'lumi_abs_err', 'lumi_unabs_err', 'Broad S/N', 'Detected in bands'
]

log_lumi_unabs = np.log10(lumi_unabs)
log_lumi_abs = np.log10(lumi_abs)

log_lumi_unabs_err = np.log10(lumi_unabs + lumi_unabs_err) - log_lumi_unabs
log_lumi_abs_err = np.log10(lumi_abs + lumi_abs_err) - log_lumi_abs


rows = zip(
    x, y,
    flux_abs, flux_unabs,
    flux_abs_err, flux_unabs_err,
    log_lumi_abs, log_lumi_unabs,
    log_lumi_abs_err, log_lumi_unabs_err, SNR, energy
)


with open('resultats_xray_NGC0628.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(rows)
